In [1]:
from hpc_utils import * 
import pickle


df_chile = pd.read_parquet("data/df_chile.parquet.gzip")
df_rome  = pd.read_parquet("data/df_rome.parquet.gzip")
df_flor  = pd.read_parquet("data/df_flor.parquet.gzip")
df_italy = pd.read_parquet("data/df_italy.parquet.gzip")


binary_target= 'Alpha-Fet'


def launch_to_ray(df_train,binary_target, save_to ,n_trials=50 ,Independent_testset = None, df_test = None):
    ray.shutdown()

    ray_df_train = ray.put(df_train)
    ray_df_test  = ray.put(df_test)
    features : list[str] = df_train.columns.tolist()
    features.remove(binary_target)
    #return ray_df_train, ray_df_test, features
    
    studies = make_multiple_studies(
        ray_df_train, 
        features = features,
        targets = [binary_target],
        n_trials=n_trials,
        Independent_testset = Independent_testset,
        Independent_testset_df = ray_df_test
    )

    S = ray.get(studies)
    with open(save_to + '.pickle', 'wb') as handle:
        pickle.dump(S, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    
    ray.shutdown()



launch_to_ray(df_train = df_chile , binary_target= binary_target, save_to = "studies_1k_chile", n_trials = 3, Independent_testset = False)
launch_to_ray(df_train = df_chile , binary_target= binary_target, save_to = "studies_1k_rome", n_trials =  3, Independent_testset = True, df_test = df_rome)
launch_to_ray(df_train = df_chile , binary_target= binary_target,save_to= "studies_1k_flor" , n_trials =   3,Independent_testset = True, df_test = df_flor)
launch_to_ray(df_train = df_chile , binary_target= binary_target,save_to= "studies_1k_italy" , n_trials =  3,Independent_testset = True, df_test = df_italy)

2023-06-08 20:29:37,491	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(make_a_study pid=927385) [I 2023-06-08 20:29:39,082] A new study created in memory with name: suac (Alpha-Fet)
(make_a_study pid=927390) [I 2023-06-08 20:29:39,083] A new study created in memory with name: age at diagnosis (months) (Alpha-Fet)
(make_a_study pid=927392) [I 2023-06-08 20:29:39,075] A new study created in memory with name: ntbc levels (dbs) (Alpha-Fet)
(make_a_study pid=927378) [I 2023-06-08 20:29:39,092] A new study created in memory with name: ntbc dosis mg/kg/day (Alpha-Fet)
(make_a_study pid=927378) [I 2023-06-08 20:29:54,663] Trial 3 finished with values: [0.8267045454545454, 0.034453608095645905] and parameters: {'seed': 2756, 'max_depth': 4, 'eta': 0.28657672275176427, 'subsample': 0.2815268736977331, 'lambda': 0.5551142106419572, 'alpha': 0.19001923346476357, 'scale_pos_weight': 0.4607955853068524, 'kfold_splits': 2}. 
(make_a_study pid=927390) [I 20

KeyboardInterrupt: 

(make_a_study pid=927392) [I 2023-06-08 20:30:18,298] Trial 30 finished with values: [0.8948863636363636, 0.007245856337249279] and parameters: {'seed': 7448, 'max_depth': 3, 'eta': 0.09142054530515994, 'subsample': 0.282127574569365, 'lambda': 0.4718810535573472, 'alpha': 0.43861649931910973, 'scale_pos_weight': 3.998438774504325, 'kfold_splits': 5}.  [repeated 34x across cluster]


In [ ]:
# %%time

# studies = make_multiple_studies(
#     ray_df_train, 
#     features = features,
#     targets = [binary_target],
#     n_trials=50,
#     Independent_testset = False,
#     Independent_testset_df = ray_df_test
# )

# studies = ray.get(studies)